Scrape Questions and metadata


In [1]:
from leetscrape import GetQuestionsList, GetQuestion

Query the question list


In [3]:
ls = GetQuestionsList()
ls.scrape()  # Scrape the list of questions
ls.to_csv(directory="../data_test/")  # Save the scraped tables to a directory


Scraping companies ... Done
Scraping questions list ... Done
Extracting question topics ... Done
Getting Categories ... Done
Scraping Topic Tags ... Done
Extracting question category ... Done


Query Individual question info such as the body, test cases, constraints, hints, code stubs, etc.


In [9]:
qi = GetQuestion(titleSlug="max-consecutive-ones-ii")
qi.scrape()

D:\code\python\LeetScrape\src\leetscrape\question.py:106: UserWarning: This questions is only for paid Leetcode subscribers.
  warnings.warn("This questions is only for paid Leetcode subscribers.")
D:\code\python\LeetScrape\src\leetscrape\question.py:143: UserWarning: This questions is only for paid Leetcode subscribers.
  warnings.warn("This questions is only for paid Leetcode subscribers.")


487. max-consecutive-ones-ii
This questions is only for paid Leetcode subscribers.SimilarQuestions: [485, 1004, 2155]

Generate questions and test cases

In [10]:
from leetscrape import GenerateCodeStub

fcs = GenerateCodeStub(qid=487)
fcs.generate()

Generating code stub for 487. max-consecutive-ones-ii
Code stub save to q_0487_maxConsecutiveOnesIi.py


D:\code\python\LeetScrape\src\leetscrape\question.py:106: UserWarning: This questions is only for paid Leetcode subscribers.
  warnings.warn("This questions is only for paid Leetcode subscribers.")
D:\code\python\LeetScrape\src\leetscrape\question.py:143: UserWarning: This questions is only for paid Leetcode subscribers.
  warnings.warn("This questions is only for paid Leetcode subscribers.")


Extract solutions from py file and upload them to the database

In [3]:
from leetscrape import ExtractSolutions

In [4]:
es = ExtractSolutions("../solutions/q_0009_palindromeNumber.py")
sols = es.extract()
es.to_mdx("../solutions/q_0009_palindromeNumber.md")